<a href="https://colab.research.google.com/github/julialromero/regional-representations-graph-model/blob/master/dataset/earth_engine/download_landsat_imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip install pydrive

In [1]:
import sys
import os
import geemap
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

In [6]:
def get_img(geoid, tracts):
    feat = tracts.filter(ee.Filter.eq('GEOID', geoid)).first()
    roi = feat.geometry()

    collection = (
      ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')  # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1
      .filterBounds(roi)
      .filter(ee.Filter.contains('.geo', roi))
      .filterDate('2020-01-01', '2023-01-01')
      .sort('CLOUD_COVER')
    )
    top = collection.limit(1)

    # take the median pixel value; clip to the census geometry 
    # composite = top.median().clip(roi)
#     print(roi)
    img = top.first().clip(roi)
    return img#, geoid
    

def createDir(dirPath):
  try:
    os.makedirs(dirPath)
  except:
    pass

def init_dir(subdir, fn):
  if os.path.exists(subdir):
      if fn in os.listdir(subdir):
#         print('pass')
        return True
     
  if not os.path.exists(subdir):
      print('creating directory')
      createDir(subdir)
      return True

In [7]:
import ee
# ee.Authenticate()
ee.Initialize()

In [15]:
tract_data_dir = r'../Tracts/nyc_metro_boundaries/nyc_metro_boundaries.shp'
temp_tract_data_dir = "../Tracts/nyc_metro_boundaries/temp.shp"
temp_envelope_data_dir = "../Tracts/nyc_metro_boundaries/temp_envelope.shp"

df = gpd.read_file(tract_data_dir)
df.rename(columns={'GEOID_TRAC': 'GEOID'}, inplace=True)  
# print(df.head())
shapefile = df[['GEOID', 'geometry']] 
# shapefile['geometry'] = shapefile['geometry'].envelope

In [45]:
# Map = geemap.Map()
# Map.setCenter(-74.0060, 40.7128, 8)
# visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
# Map.addLayer(top, visParams, 'tracts')
# Map

In [41]:
save_dir = 'download_landsat_images'
count = 0
for interval in range(0, shapefile.shape[0], 20): 
    try:
        sub = shapefile.iloc[interval:interval+20]
        sub.to_file(temp_tract_data_dir)
    except:
        sub = shapefile.iloc[interval:-1]
        sub.to_file(temp_tract_data_dir)
    tracts = geemap.shp_to_ee(temp_tract_data_dir)

    geoids = tracts.aggregate_array('GEOID')
    result = geoids.map(get_img)
    for i in range(0, sub.shape[0]):
#         print('-')
#         polygon = shapefile.iloc[i][1]
        geoid = shapefile.iloc[interval+i]['GEOID']
        
        subdir = f"{save_dir}/{geoid}"
        
        #### CHANGE THIS LINE ####
        filename = f"{subdir}/least_cloudy_rectangle_highres.tif"
        
        if os.path.exists(filename):
#             print(f'continue: {filename}')
            continue
        init_dir(subdir, filename)
                 
        img = ee.Image(result.get(i))
        img = img.select(['B2', 'B3', 'B4', 'B5'])
                
        try:
            if img.projection().nominalScale().getInfo() != 30:
                count +=1
                continue
        except:
            print('--->Error getting projection. Passing...')
        
        geemap.ee_export_image(
                img, filename=filename, file_per_band=False
            )
        print('---')

#         if not os.path.exists(filename):
#             img = img.select(['B1', 'B2', 'B3','B4', 'B5', 'B6', 'B7', 'B8'])
#             geemap.ee_export_image(
#                     img, filename=filename, file_per_band=False
#                 )
            
#         if not os.path.exists(filename):
#             img = img.select(['B1', 'B2', 'B3','B4', 'B5', 'B6', 'B7'])
#             geemap.ee_export_image(
#                     img, filename=filename, file_per_band=False
#                 )
#         if not os.path.exists(filename):
#             img = img.select(['B1', 'B2', 'B3','B4', 'B5', 'B6'])
#             geemap.ee_export_image(
#                     img, filename=filename, file_per_band=False
#                 )
            
#         if not os.path.exists(filename):
#             img = img.select(['B1', 'B2', 'B3','B4', 'B5'])
#             geemap.ee_export_image(
#                     img, filename=filename, file_per_band=False
#                 )
            
#         if not os.path.exists(filename):
#             img = img.select(['B1', 'B2', 'B3','B4'])
#             geemap.ee_export_image(
#                     img, filename=filename, file_per_band=False
#                 )

        
#         if os.path.exists(filename): 
#             print('saved')
      

    print(f'-----DONE {interval} - {interval + sub.shape[0]}')
print(f'Number of images with scale not at 30 m resolution: {count}')
    

-----DONE 0 - 20
-----DONE 20 - 40
-----DONE 40 - 60
-----DONE 60 - 80
-----DONE 80 - 100
-----DONE 100 - 120
-----DONE 120 - 140
-----DONE 140 - 160
-----DONE 160 - 180
-----DONE 180 - 200
-----DONE 200 - 220
-----DONE 220 - 240
-----DONE 240 - 260
-----DONE 260 - 280
-----DONE 280 - 300
-----DONE 300 - 320
-----DONE 320 - 340
-----DONE 340 - 360
-----DONE 360 - 380
-----DONE 380 - 400
-----DONE 400 - 420
-----DONE 420 - 440
-----DONE 440 - 460
-----DONE 460 - 480
-----DONE 480 - 500
-----DONE 500 - 520
-----DONE 520 - 540
-----DONE 540 - 560
-----DONE 560 - 580
-----DONE 580 - 600
-----DONE 600 - 620
-----DONE 620 - 640
-----DONE 640 - 660
-----DONE 660 - 680
-----DONE 680 - 700
-----DONE 700 - 720
-----DONE 720 - 740
-----DONE 740 - 760
-----DONE 760 - 780
-----DONE 780 - 800
-----DONE 800 - 820
-----DONE 820 - 840
-----DONE 840 - 860
-----DONE 860 - 880
-----DONE 880 - 900
-----DONE 900 - 920
-----DONE 920 - 940
-----DONE 940 - 960
-----DONE 960 - 980
-----DONE 980 - 1000
-----DONE

Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/42071011805/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/42071014401/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/42071013701/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/42071011701/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948

Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/42029307700/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/42079217200/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/42079215800/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/42079216100/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948

Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/42013101100/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/42013101400/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/42071000500/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/42071013101/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948

Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/42071000600/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/42071000700/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/42071000800/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/42071000900/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948

...

Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34039031802/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34035053801/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34017019100/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34017019300/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948

Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34019011800/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34017017000/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34017014501/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34017014502/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948

Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34023003101/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34035053905/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34035050704/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34005700303/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948

Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34005702206/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34005701105/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34005702204/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34005701001/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948

Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34023007909/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34023007910/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34027041200/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948/region_representation_learning/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34027041500/least_cloudy_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /home/juro4948

In [37]:
geoid

'0'

In [18]:
shapefile.loc[shapefile.GEOID == geoid].to_file(temp_tract_data_dir)
tracts = geemap.shp_to_ee(temp_envelope_data_dir)

In [46]:
feat = envelope_tracts.filter(ee.Filter.eq('GEOID', geoid)).first()
roi = feat.geometry()

In [47]:
roi

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Element.geometry",
    "arguments": {
      "feature": {
        "functionInvocationValue": {
          "functionName": "Collection.first",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection",
                      "arguments": {
                        "features": {
                          "arrayValue": {
                            "values": [
                              {
                                "functionInvocationValue": {
                                  "functionName": "Feature",
                                  "arguments": {
                                    "geometry": {
                                      "functionInvocationValue": {
                                        "functionName": "GeometryConstructors.Polygon",
                                        "arguments": {
                                          "coordinates": {
                                            "constantValue": [
                                              [
                                                [
                                                  -73.431919,
                                                  40.534265
                                                ],
                                                [
                                                  -73.431919,
                                                  41.304483
                                                ],
                                                [
                                                  -71.777491,
                                                  41.304483
                                                ],
                                                [
                                                  -71.777491,
                                                  40.534265
                                                ],
                                                [
                                                  -73.431919,
                                                  40.534265
                                                ]
                                              ]
                                            ]
                                          },
                                          "geodesic": {
                                            "constantValue": false
                                          }
                                        }
                                      }
                                    },
                                    "metadata": {
                                      "constantValue": {
                                        "GEOID": "36103990100"
                                      }
                                    }
                                  }
                                }
                              }
                            ]
                          }
                        }
                      }
                    }
                  },
                  "filter": {
                    "functionInvocationValue": {
                      "functionName": "Filter.equals",
                      "arguments": {
                        "leftField": {
                          "constantValue": "GEOID"
                        },
                        "rightValue": {
                          "constantValue": "36103990100"
                        }
                      }
                    }
                  }
                }
              }
            }
          }
        }
      }
    }
  }
})

In [60]:
# img = get_img(geoid)
collection = (
      ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')  # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1
      .filterBounds(roi)
#       .filter(ee.Filter.contains('.geo', roi))
      .filterDate('2020-01-01', '2023-01-01')
      .sort('CLOUD_COVER')
    )
top = collection.limit(5)

In [61]:
top

In [62]:
# plot all
Map = geemap.Map()
Map.setCenter(-74.0060, 40.7128, 8)
visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
Map.addLayer(top, visParams, 'Landsat8')
Map.addLayer(roi, {}, 'Census Tracts')
Map

Map(center=[40.7128, -74.006], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [44]:
envelope_tracts

In [45]:
tracts

In [42]:
[r[-2]]

['36103990100']

In [43]:
def get_regions_to_retry():
    l = []
    counter=0
    for root, dirs, files in os.walk("download_landsat_images/"):
        if '.ipynb_checkpoints' in root:
            continue
        for f in files:
            if f == 'least_cloudy_rectangle_highres.tif':
                counter+=1
        if 'least_cloudy_rectangle_highres.tif' not in files:
            l.append(root)
#             print(root)
            
    l.remove('download_landsat_images/')
    al = len(os.listdir("download_landsat_images/"))
    print(f'Num folders total: {al}')
    print(f'Num target files total: {counter}')
    print()
    l = [item[-11:] for item in l]
    return l
    
r = get_regions_to_retry()

for geoid in [r[-2]]:
    shapefile.loc[shapefile.GEOID == geoid].to_file(temp_tract_data_dir)
    tracts = geemap.shp_to_ee(temp_tract_data_dir)
    
    envelope_shp = shapefile.loc[shapefile.GEOID == geoid]
    envelope_shp['geometry'] = envelope_shp['geometry'].envelope
    envelope_shp.to_file(temp_envelope_data_dir)
    envelope_tracts = geemap.shp_to_ee(temp_envelope_data_dir)
    
    img = get_img(geoid, envelope_tracts)
    save_dir = 'download_landsat_images'
    subdir = f"{save_dir}/{geoid}"
    filename = f"{subdir}/least_cloudy_rectangle_highres.tif"

#     img = ee.Image(result.get(i))
    img = img.select(['B2', 'B3', 'B4', 'B5'])

    try:
        if img.projection().nominalScale().getInfo() != 30:
            count +=1
            continue
    except:
        print('--->Error getting projection. Passing...')

    geemap.ee_export_image(
            img, filename=filename, file_per_band=False)


    print('--\n')

Num folders total: 8232
Num target files total: 8226

--->Error getting projection. Passing...
Generating URL ...
An error occurred while downloading.
Image.clip: Parameter 'input' is required.
--



In [ ]:
# # plot all
# Map = geemap.Map()
# Map.setCenter(-74.0060, 40.7128, 8)
# visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
# Map.addLayer(collection.first(), visParams, 'Landsat8')
# Map

# Test package



In [22]:
cld

In [23]:
img

In [27]:
# my scenario
ee.Initialize()

# aoi = ee.Geometry(Point([-48.4, -23]))
# aoi = shapefile.iloc[0][1]
# img = ee.Image(result.get(i))
        
        
# clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterDate('2019-11-25', '2019-11-28').filterBounds(aoi)


Map = geemap.Map()
Map.setCenter(-74.0060, 40.7128, 8)
visParams = {'bands': ['B2'], 'min': 0, 'max': 0.2}
Map.addLayer(img, visParams, 'band 2')
Map


# Map = geemap.Map(center=(-23, -48.4), zoom=9)
# Map.addLayer(img, {'min': 0, 'max': 100, 'palette': ['purple', 'yellow']}, 'Cloud Probability', True, 1)
# Map

Map(center=[40.7128, -74.006], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

In [21]:
ee.Initialize()

aoi = ee.Geometry(Point([-48.4, -23]))

clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterDate('2019-11-25', '2019-11-28').filterBounds(aoi)

# img = s2.first()
cld = clouds.first()

Map = geemap.Map(center=(-23, -48.4), zoom=9)
Map.addLayer(cld, {'min': 0, 'max': 100, 'palette': ['purple', 'yellow']}, 'Cloud Probability', True, 1)
Map

Map(center=[-23, -48.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

## Scratchwork

In [ ]:
import tqdm


stats = pd.DataFrame(columns = [
    'geoid',
    'max',
 'mean',
 'min',
 'sample_sd',
 'sample_var',
 'sum',
 'sum_sq',
 'total_count',
 'total_sd',
 'total_var',
 'valid_count',
 'weight_sum',
 'weighted_sum'])

img_list = ee.List([])


def save_images(geoid):
    print(geoid_list)
    for geoid in geoid_list:
        print(geoid)
    save_dir = 'download_landsat_images/'
    subdir = f"{save_dir}{geoid}"


    if os.path.exists(subdir):
        if len(os.listdir(subdir)) >= 10:
#               print('pass')
          return True
        else:
#               print(f'Redoing: {subdir}')
            pass

    if not os.path.exists(subdir):
#             print('creating directory')
        createDir(subdir)

    
    geemap.ee_export_image(img, filename=f'{subdir}/least_CC.tif', region=roi, scale=90)

    # for image collection stats 
#         temp = pd.DataFrame(data=img.toArray().getInfo()).T
#         temp.columns=temp.keys().getInfo()
#         temp['geoid'] = geoid

#         stats = pd.concat([stats, temp], axis=0)
#         stats.to_csv('save_dir/_CC_stats.csv', index=False)


#         for img_idx in top.aggregate_array('system:index').getInfo():
#             img = ee.Image('LANDSAT/LC08/C02/T1_TOA/' + img_idx)
#             filename = f"{subdir}/{img_idx}.tif"

#             new_img = img.clip(roi)

#             geemap.ee_export_image(
#                 new_img, filename=filename, scale=90, region=roi, file_per_band=False
#             )

    return 

In [ ]:
# plot all
Map = geemap.Map()
Map.setCenter(-74.0060, 40.7128, 8)
visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
Map.addLayer(collection.first(), visParams, 'Landsat8')
# Map.addLayer(roi, {}, 'Census Tracts')
Map

In [ ]:
Map = geemap.Map()
Map.setCenter(-74.0060, 40.7128, 8)
visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
Map.addLayer(new_img, visParams, 'Landsat8')
# Map.addLayer(roi, {}, 'Census Tracts')
Map

In [ ]:
save_dir = 'download_landsat_images/'
def Count_files_in_subd():
    l = []
    
    for root, dirs, files in os.walk(save_dir):
        # print("{} in {}".format(len(files), root))
        l.append(len(files))
#         if len(files) == 0:
#           print(dirs)
    al = len(os.listdir(save_dir))
    print(f'Num folders total: {al}')
    print(f'Num folders with 0 files: {l.count(0)}')
    print(f'Num folders with >0 files: {al - l.count(0)}')
    print()
Count_files_in_subd()